In [3]:
import pandas as pd
import pyodbc
from datetime import datetime
import os

In [4]:
server = 'tcp:45.131.109.84'
database = 'EnderleSolutions'
username = 'SA'
password = 'Mila2015'
conn = pyodbc.connect(
     'DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
cursor = conn.cursor()

In [5]:
path = "../../../DATEN/Excel-Export/Auswertung_2021/"
quarter = False
start = datetime(2021, 1, 1)
end = datetime(2021, 12, 31)

In [6]:
if not os.path.exists(path):
    os.makedirs(path)

In [7]:
if not quarter:
    movements = pd.read_sql_query(f"""
        SELECT w.[RechnungsNummer], w.[ArtikelNummer], w.[Anzahl], w.[Verkaufspreis], 
        a.[Bezeichnung], a.[Art] as ArtikelArt, r.[Art] as RechnungsArt, r.[Lieferant], 
        r.[BetragNetto], r.[BetragBrutto], r.[Rechnungsdatum], r.[Zahlungsdatum], r.[Versandkosten] 
            FROM [EnderleSolutions].[dbo].[Wareneingang_Artikel] w 
                JOIN [EnderleSolutions].[dbo].[Artikelnummern_Lieferanten] a ON w.ArtikelNummer = a.ArtikelNummer 
                JOIN [EnderleSolutions].[dbo].[Rechnung_Lieferant] r ON w.RechnungsNummer = r.RechnungsNummer
            WHERE r.Zahlungsdatum >= '{start}' AND r.Zahlungsdatum <= '{end}';
    """, conn)
else:
    movements = pd.read_sql_query(f"""
        SELECT w.[RechnungsNummer], w.[ArtikelNummer], w.[Anzahl], w.[Verkaufspreis], 
        a.[Bezeichnung], a.[Art] as ArtikelArt, r.[Art] as RechnungsArt, r.[Lieferant], 
        r.[BetragNetto], r.[BetragBrutto], r.[Rechnungsdatum], r.[Zahlungsdatum], r.[Versandkosten] 
            FROM [EnderleSolutions].[dbo].[Wareneingang_Artikel] w 
                JOIN [EnderleSolutions].[dbo].[Artikelnummern_Lieferanten] a ON w.ArtikelNummer = a.ArtikelNummer 
                JOIN [EnderleSolutions].[dbo].[Rechnung_Lieferant] r ON w.RechnungsNummer = r.RechnungsNummer
            AND r.Rechnungsdatum >= '{start}' AND r.Rechnungsdatum <= '{end}';
    """, conn)

movements.head()

C:\Users\Moritz\AppData\Local\Temp\ipykernel_7220\3454302242.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movements = pd.read_sql_query(f"""


,RechnungsNummer,ArtikelNummer,Anzahl,Verkaufspreis,Bezeichnung,ArtikelArt,RechnungsArt,Lieferant,BetragNetto,BetragBrutto,Rechnungsdatum,Zahlungsdatum,Versandkosten
0,R22101814,03010302000,1.0,42.86,"Tacho D60 M18 100km/h elf k=1,0",26 Wareneinkauf,None,Pulsotronic-Anlagentechnik,47.90,57.00,2021-05-12,2021-05-04,5.04
1,103150,083 860 570,10.0,9.90,BREMSLICHTSCHALTER,26 Wareneinkauf,None,Hönert,103.95,123.70,2021-03-23,2021-03-25,4.95
2,108044,083 860 570,10.0,9.90,BREMSLICHTSCHALTER,26 Wareneinkauf,None,Hönert,103.95,123.70,2021-08-09,2021-08-16,4.95
3,RE-2021-02-6507,100117,20.0,0.88,Hülse,26 Wareneinkauf,None,ARC Reuter,122.90,146.25,2021-02-08,2021-02-26,5.90
4,202199389,100779,10.0,1.40,Feder Auspuff LTH JL-Länge 65mm,26 Wareneinkauf,None,LTH,84.46,100.51,2021-08-19,2021-08-19,5.46


In [8]:
movements["Steuersatz"] = (movements["BetragBrutto"] - movements["BetragNetto"]) / movements["BetragNetto"]
movements["Steuersatz"] = movements["Steuersatz"].round(2)
movements.head(2)

,RechnungsNummer,ArtikelNummer,Anzahl,Verkaufspreis,Bezeichnung,ArtikelArt,RechnungsArt,Lieferant,BetragNetto,BetragBrutto,Rechnungsdatum,Zahlungsdatum,Versandkosten,Steuersatz
0,R22101814,03010302000,1.0,42.86,"Tacho D60 M18 100km/h elf k=1,0",26 Wareneinkauf,None,Pulsotronic-Anlagentechnik,47.90,57.0,2021-05-12,2021-05-04,5.04,0.19
1,103150,083 860 570,10.0,9.90,BREMSLICHTSCHALTER,26 Wareneinkauf,None,Hönert,103.95,123.7,2021-03-23,2021-03-25,4.95,0.19


In [9]:
# create a column "Art" which is the ArtikelArt if it is not null, otherwise the RechnungsArt
movements["Art"] = movements["ArtikelArt"].where(movements["ArtikelArt"].notnull(), movements["RechnungsArt"])
# drop the columns ArtikelArt and RechnungsArt
movements = movements.drop(columns=["ArtikelArt", "RechnungsArt"])
movements.head(5)

,RechnungsNummer,ArtikelNummer,Anzahl,Verkaufspreis,Bezeichnung,Lieferant,BetragNetto,BetragBrutto,Rechnungsdatum,Zahlungsdatum,Versandkosten,Steuersatz,Art
0,R22101814,03010302000,1.0,42.86,"Tacho D60 M18 100km/h elf k=1,0",Pulsotronic-Anlagentechnik,47.90,57.00,2021-05-12,2021-05-04,5.04,0.19,26 Wareneinkauf
1,103150,083 860 570,10.0,9.90,BREMSLICHTSCHALTER,Hönert,103.95,123.70,2021-03-23,2021-03-25,4.95,0.19,26 Wareneinkauf
2,108044,083 860 570,10.0,9.90,BREMSLICHTSCHALTER,Hönert,103.95,123.70,2021-08-09,2021-08-16,4.95,0.19,26 Wareneinkauf
3,RE-2021-02-6507,100117,20.0,0.88,Hülse,ARC Reuter,122.90,146.25,2021-02-08,2021-02-26,5.90,0.19,26 Wareneinkauf
4,202199389,100779,10.0,1.40,Feder Auspuff LTH JL-Länge 65mm,LTH,84.46,100.51,2021-08-19,2021-08-19,5.46,0.19,26 Wareneinkauf


In [10]:
# sort the dataframe by Rechnungsnummer and Rechnungsdatum
movements = movements.sort_values(by=["Rechnungsdatum", "RechnungsNummer"])
movements.head(10)

,RechnungsNummer,ArtikelNummer,Anzahl,Verkaufspreis,Bezeichnung,Lieferant,BetragNetto,BetragBrutto,Rechnungsdatum,Zahlungsdatum,Versandkosten,Steuersatz,Art
36,RE20038159,131232,1.0,7.76,Gasschieber -DELLORTO PHBG- (40),SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,Kleinmaterial
151,RE20038159,3332376,3.0,20.62,"Ansaugbalg -OEM QUALITÄT- Vespa Wideframe VN, ...",SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,26 Wareneinkauf
176,RE20038159,3820008FPM,5.0,5.27,"Wellendichtring 27x42x10mm -FPM, Doppellippe- ...",SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,26 Wareneinkauf
249,RE20038159,7671252,1.0,7.90,"Nadelset -BGM PRO- Dellorto PHBG (W3, W4, W5, ...",SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,Kleinmaterial
250,RE20038159,7671257,1.0,6.97,Nebendüsenset -BGM PRO für Dellorto (Typ: 1488...,SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,Kleinmaterial
686,RE20038159,BM1081,5.0,13.73,Chokezugclip für Chokezug Polini CP Vergaser,SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,26 Wareneinkauf
689,RE20038159,C11636,1.0,5.49,Kabelchoke-Kit -DELLORTO- PHBG,SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,26 Wareneinkauf
241,RE-A-20117862,7300108,2.0,24.60,Stoßdämpferfeder vorne -STO l=155mm- Vespa 125...,SC,666.03,772.60,2020-11-18,2021-01-29,1.26,0.16,26 Wareneinkauf
258,RE-A-20117862,7671486,3.0,53.83,Stoßdämpfer vorne -CIF- Vespa Largeframe Rall...,SC,666.03,772.60,2020-11-18,2021-01-29,1.26,0.16,26 Wareneinkauf
302,RE-A-20117862,7675717N,2.0,48.12,Stoßdämpferfeder vorne -NAGY BLECHROLLER l=14...,SC,666.03,772.60,2020-11-18,2021-01-29,1.26,0.16,26 Wareneinkauf


In [11]:
for invoice_num in movements["RechnungsNummer"].unique():
    if movements[movements["RechnungsNummer"] == invoice_num]["Versandkosten"].iloc[0] != 0:
        tmp = pd.DataFrame({
            "RechnungsNummer": [invoice_num],
            "ArtikelNummer": [""],
            "Anzahl": [1],
            "Verkaufspreis": [movements[movements["RechnungsNummer"] == invoice_num]["Versandkosten"].iloc[0]],
            "Bezeichnung":  [f"Versandkosten für Rechnung {invoice_num}"],
            "BetragNetto": [None],
            "BetragBrutto": [None],
            "Rechnungsdatum": [movements[movements["RechnungsNummer"] == invoice_num]["Rechnungsdatum"].iloc[0]],
            "Zahlungsdatum": [movements[movements["RechnungsNummer"] == invoice_num]["Zahlungsdatum"].iloc[0]],
            "Steuersatz": [movements[movements["RechnungsNummer"] == invoice_num]["Steuersatz"].iloc[0]],
            "Art": [movements[movements["RechnungsNummer"] == invoice_num]["Art"].iloc[0]],
            "Lieferant": [movements[movements["RechnungsNummer"] == invoice_num]["Lieferant"].iloc[0]]
        })
        movements = pd.concat([movements, tmp], ignore_index=True, axis=0)

movements.head(10)

,RechnungsNummer,ArtikelNummer,Anzahl,Verkaufspreis,Bezeichnung,Lieferant,BetragNetto,BetragBrutto,Rechnungsdatum,Zahlungsdatum,Versandkosten,Steuersatz,Art
0,RE20038159,131232,1.0,7.76,Gasschieber -DELLORTO PHBG- (40),SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,Kleinmaterial
1,RE20038159,3332376,3.0,20.62,"Ansaugbalg -OEM QUALITÄT- Vespa Wideframe VN, ...",SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,26 Wareneinkauf
2,RE20038159,3820008FPM,5.0,5.27,"Wellendichtring 27x42x10mm -FPM, Doppellippe- ...",SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,26 Wareneinkauf
3,RE20038159,7671252,1.0,7.90,"Nadelset -BGM PRO- Dellorto PHBG (W3, W4, W5, ...",SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,Kleinmaterial
4,RE20038159,7671257,1.0,6.97,Nebendüsenset -BGM PRO für Dellorto (Typ: 1488...,SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,Kleinmaterial
5,RE20038159,BM1081,5.0,13.73,Chokezugclip für Chokezug Polini CP Vergaser,SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,26 Wareneinkauf
6,RE20038159,C11636,1.0,5.49,Kabelchoke-Kit -DELLORTO- PHBG,SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,26 Wareneinkauf
7,RE-A-20117862,7300108,2.0,24.60,Stoßdämpferfeder vorne -STO l=155mm- Vespa 125...,SC,666.03,772.60,2020-11-18,2021-01-29,1.26,0.16,26 Wareneinkauf
8,RE-A-20117862,7671486,3.0,53.83,Stoßdämpfer vorne -CIF- Vespa Largeframe Rall...,SC,666.03,772.60,2020-11-18,2021-01-29,1.26,0.16,26 Wareneinkauf
9,RE-A-20117862,7675717N,2.0,48.12,Stoßdämpferfeder vorne -NAGY BLECHROLLER l=14...,SC,666.03,772.60,2020-11-18,2021-01-29,1.26,0.16,26 Wareneinkauf


In [12]:
movements = movements.sort_values(by=["Rechnungsdatum", "RechnungsNummer"])
movements.head(10)

,RechnungsNummer,ArtikelNummer,Anzahl,Verkaufspreis,Bezeichnung,Lieferant,BetragNetto,BetragBrutto,Rechnungsdatum,Zahlungsdatum,Versandkosten,Steuersatz,Art
0,RE20038159,131232,1.0,7.76,Gasschieber -DELLORTO PHBG- (40),SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,Kleinmaterial
1,RE20038159,3332376,3.0,20.62,"Ansaugbalg -OEM QUALITÄT- Vespa Wideframe VN, ...",SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,26 Wareneinkauf
2,RE20038159,3820008FPM,5.0,5.27,"Wellendichtring 27x42x10mm -FPM, Doppellippe- ...",SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,26 Wareneinkauf
3,RE20038159,7671252,1.0,7.90,"Nadelset -BGM PRO- Dellorto PHBG (W3, W4, W5, ...",SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,Kleinmaterial
4,RE20038159,7671257,1.0,6.97,Nebendüsenset -BGM PRO für Dellorto (Typ: 1488...,SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,Kleinmaterial
5,RE20038159,BM1081,5.0,13.73,Chokezugclip für Chokezug Polini CP Vergaser,SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,26 Wareneinkauf
6,RE20038159,C11636,1.0,5.49,Kabelchoke-Kit -DELLORTO- PHBG,SC,186.24,221.63,2020-05-05,2021-12-15,1.26,0.19,26 Wareneinkauf
716,RE20038159,,1.0,1.26,Versandkosten für Rechnung RE20038159,SC,NaN,NaN,2020-05-05,2021-12-15,NaN,0.19,Kleinmaterial
7,RE-A-20117862,7300108,2.0,24.60,Stoßdämpferfeder vorne -STO l=155mm- Vespa 125...,SC,666.03,772.60,2020-11-18,2021-01-29,1.26,0.16,26 Wareneinkauf
8,RE-A-20117862,7671486,3.0,53.83,Stoßdämpfer vorne -CIF- Vespa Largeframe Rall...,SC,666.03,772.60,2020-11-18,2021-01-29,1.26,0.16,26 Wareneinkauf


In [13]:
# drop columns BetragNetto and BetragBrutto and Versandkosten
movements = movements.drop(columns=["BetragNetto", "BetragBrutto", "Versandkosten"])
movements.head(5)

,RechnungsNummer,ArtikelNummer,Anzahl,Verkaufspreis,Bezeichnung,Lieferant,Rechnungsdatum,Zahlungsdatum,Steuersatz,Art
0,RE20038159,131232,1.0,7.76,Gasschieber -DELLORTO PHBG- (40),SC,2020-05-05,2021-12-15,0.19,Kleinmaterial
1,RE20038159,3332376,3.0,20.62,"Ansaugbalg -OEM QUALITÄT- Vespa Wideframe VN, ...",SC,2020-05-05,2021-12-15,0.19,26 Wareneinkauf
2,RE20038159,3820008FPM,5.0,5.27,"Wellendichtring 27x42x10mm -FPM, Doppellippe- ...",SC,2020-05-05,2021-12-15,0.19,26 Wareneinkauf
3,RE20038159,7671252,1.0,7.90,"Nadelset -BGM PRO- Dellorto PHBG (W3, W4, W5, ...",SC,2020-05-05,2021-12-15,0.19,Kleinmaterial
4,RE20038159,7671257,1.0,6.97,Nebendüsenset -BGM PRO für Dellorto (Typ: 1488...,SC,2020-05-05,2021-12-15,0.19,Kleinmaterial


In [14]:
# add a column "MwSt. pro Stück" which is the Steuersatz * Verkaufspreis
movements["MwSt. pro Stück"] = movements["Steuersatz"] * movements["Verkaufspreis"]
movements.head(5)

,RechnungsNummer,ArtikelNummer,Anzahl,Verkaufspreis,Bezeichnung,Lieferant,Rechnungsdatum,Zahlungsdatum,Steuersatz,Art,MwSt. pro Stück
0,RE20038159,131232,1.0,7.76,Gasschieber -DELLORTO PHBG- (40),SC,2020-05-05,2021-12-15,0.19,Kleinmaterial,1.4744
1,RE20038159,3332376,3.0,20.62,"Ansaugbalg -OEM QUALITÄT- Vespa Wideframe VN, ...",SC,2020-05-05,2021-12-15,0.19,26 Wareneinkauf,3.9178
2,RE20038159,3820008FPM,5.0,5.27,"Wellendichtring 27x42x10mm -FPM, Doppellippe- ...",SC,2020-05-05,2021-12-15,0.19,26 Wareneinkauf,1.0013
3,RE20038159,7671252,1.0,7.90,"Nadelset -BGM PRO- Dellorto PHBG (W3, W4, W5, ...",SC,2020-05-05,2021-12-15,0.19,Kleinmaterial,1.5010
4,RE20038159,7671257,1.0,6.97,Nebendüsenset -BGM PRO für Dellorto (Typ: 1488...,SC,2020-05-05,2021-12-15,0.19,Kleinmaterial,1.3243


In [15]:
# add a columns Gesamtposition Brutto
movements["Gesamtposition Brutto"] = movements["Anzahl"] * movements["Verkaufspreis"] * (1 + movements["Steuersatz"])
# round to 2 decimals
movements["Gesamtposition Brutto"] = movements["Gesamtposition Brutto"].round(2)
movements.head(5)

,RechnungsNummer,ArtikelNummer,Anzahl,Verkaufspreis,Bezeichnung,Lieferant,Rechnungsdatum,Zahlungsdatum,Steuersatz,Art,MwSt. pro Stück,Gesamtposition Brutto
0,RE20038159,131232,1.0,7.76,Gasschieber -DELLORTO PHBG- (40),SC,2020-05-05,2021-12-15,0.19,Kleinmaterial,1.4744,9.23
1,RE20038159,3332376,3.0,20.62,"Ansaugbalg -OEM QUALITÄT- Vespa Wideframe VN, ...",SC,2020-05-05,2021-12-15,0.19,26 Wareneinkauf,3.9178,73.61
2,RE20038159,3820008FPM,5.0,5.27,"Wellendichtring 27x42x10mm -FPM, Doppellippe- ...",SC,2020-05-05,2021-12-15,0.19,26 Wareneinkauf,1.0013,31.36
3,RE20038159,7671252,1.0,7.90,"Nadelset -BGM PRO- Dellorto PHBG (W3, W4, W5, ...",SC,2020-05-05,2021-12-15,0.19,Kleinmaterial,1.5010,9.40
4,RE20038159,7671257,1.0,6.97,Nebendüsenset -BGM PRO für Dellorto (Typ: 1488...,SC,2020-05-05,2021-12-15,0.19,Kleinmaterial,1.3243,8.29


In [16]:
# add a column "Gesamtposition Netto"
movements["Gesamtposition Netto"] = movements["Anzahl"] * movements["Verkaufspreis"]
# round to 2 decimals
movements["Gesamtposition Netto"] = movements["Gesamtposition Netto"].round(2)
movements.head(5)

,RechnungsNummer,ArtikelNummer,Anzahl,Verkaufspreis,Bezeichnung,Lieferant,Rechnungsdatum,Zahlungsdatum,Steuersatz,Art,MwSt. pro Stück,Gesamtposition Brutto,Gesamtposition Netto
0,RE20038159,131232,1.0,7.76,Gasschieber -DELLORTO PHBG- (40),SC,2020-05-05,2021-12-15,0.19,Kleinmaterial,1.4744,9.23,7.76
1,RE20038159,3332376,3.0,20.62,"Ansaugbalg -OEM QUALITÄT- Vespa Wideframe VN, ...",SC,2020-05-05,2021-12-15,0.19,26 Wareneinkauf,3.9178,73.61,61.86
2,RE20038159,3820008FPM,5.0,5.27,"Wellendichtring 27x42x10mm -FPM, Doppellippe- ...",SC,2020-05-05,2021-12-15,0.19,26 Wareneinkauf,1.0013,31.36,26.35
3,RE20038159,7671252,1.0,7.90,"Nadelset -BGM PRO- Dellorto PHBG (W3, W4, W5, ...",SC,2020-05-05,2021-12-15,0.19,Kleinmaterial,1.5010,9.40,7.90
4,RE20038159,7671257,1.0,6.97,Nebendüsenset -BGM PRO für Dellorto (Typ: 1488...,SC,2020-05-05,2021-12-15,0.19,Kleinmaterial,1.3243,8.29,6.97


In [17]:
# add a column "MwSt. Gesamt"
movements["MwSt. Gesamt"] = movements["Anzahl"] * movements["Verkaufspreis"] * movements["Steuersatz"]
# round to 2 decimals
movements["MwSt. Gesamt"] = movements["MwSt. Gesamt"].round(2)
movements.head(5)

,RechnungsNummer,ArtikelNummer,Anzahl,Verkaufspreis,Bezeichnung,Lieferant,Rechnungsdatum,Zahlungsdatum,Steuersatz,Art,MwSt. pro Stück,Gesamtposition Brutto,Gesamtposition Netto,MwSt. Gesamt
0,RE20038159,131232,1.0,7.76,Gasschieber -DELLORTO PHBG- (40),SC,2020-05-05,2021-12-15,0.19,Kleinmaterial,1.4744,9.23,7.76,1.47
1,RE20038159,3332376,3.0,20.62,"Ansaugbalg -OEM QUALITÄT- Vespa Wideframe VN, ...",SC,2020-05-05,2021-12-15,0.19,26 Wareneinkauf,3.9178,73.61,61.86,11.75
2,RE20038159,3820008FPM,5.0,5.27,"Wellendichtring 27x42x10mm -FPM, Doppellippe- ...",SC,2020-05-05,2021-12-15,0.19,26 Wareneinkauf,1.0013,31.36,26.35,5.01
3,RE20038159,7671252,1.0,7.90,"Nadelset -BGM PRO- Dellorto PHBG (W3, W4, W5, ...",SC,2020-05-05,2021-12-15,0.19,Kleinmaterial,1.5010,9.40,7.90,1.50
4,RE20038159,7671257,1.0,6.97,Nebendüsenset -BGM PRO für Dellorto (Typ: 1488...,SC,2020-05-05,2021-12-15,0.19,Kleinmaterial,1.3243,8.29,6.97,1.32


In [18]:
import os

def exist(year, invoicer, inv_num) -> tuple:
    return (
        os.path.exists(f"../../../DATEN/Eingangsrechnungen/{year}/{invoicer.replace(r'&', '_')}/{inv_num.replace(r'&', '_')}/rechnung.pdf"),
        os.path.exists(f"../../../DATEN/Eingangsrechnungen/{year}/{invoicer.replace(r'&', '_')}/{inv_num.replace(r'&', '_')}/zahlung.pdf"),
    )

def paths(year, invoicer, inv_num):
    path = f"/{invoicer.replace(r'&', '_')}/{inv_num.replace(r'&', '_')}/"
    return (
        f'=HYPERLINK("./Eingangsrechnungen{path}rechnung.pdf", "öffnen")' if exist(year, invoicer, inv_num)[0] else "---",
        f'=HYPERLINK("./Eingangsrechnungen{path}zahlung.pdf", "öffnen")' if exist(year, invoicer, inv_num)[1] else "---",
    )

In [19]:
# create a column "Rechnung" which is the path to the invoice
movements["Rechnung"] = movements.apply(lambda x: paths(x["Rechnungsdatum"].year, str(x["Lieferant"]), str(x["RechnungsNummer"]))[0], axis=1)
# create a column "Zahlung" which is the path to the payment
movements["Zahlung"] = movements.apply(lambda x: paths(x["Rechnungsdatum"].year, str(x["Lieferant"]), str(x["RechnungsNummer"]))[1], axis=1)
movements.head(5)

,RechnungsNummer,ArtikelNummer,Anzahl,Verkaufspreis,Bezeichnung,Lieferant,Rechnungsdatum,Zahlungsdatum,Steuersatz,Art,MwSt. pro Stück,Gesamtposition Brutto,Gesamtposition Netto,MwSt. Gesamt,Rechnung,Zahlung
0,RE20038159,131232,1.0,7.76,Gasschieber -DELLORTO PHBG- (40),SC,2020-05-05,2021-12-15,0.19,Kleinmaterial,1.4744,9.23,7.76,1.47,"=HYPERLINK(""./Eingangsrechnungen/SC/RE20038159...","=HYPERLINK(""./Eingangsrechnungen/SC/RE20038159..."
1,RE20038159,3332376,3.0,20.62,"Ansaugbalg -OEM QUALITÄT- Vespa Wideframe VN, ...",SC,2020-05-05,2021-12-15,0.19,26 Wareneinkauf,3.9178,73.61,61.86,11.75,"=HYPERLINK(""./Eingangsrechnungen/SC/RE20038159...","=HYPERLINK(""./Eingangsrechnungen/SC/RE20038159..."
2,RE20038159,3820008FPM,5.0,5.27,"Wellendichtring 27x42x10mm -FPM, Doppellippe- ...",SC,2020-05-05,2021-12-15,0.19,26 Wareneinkauf,1.0013,31.36,26.35,5.01,"=HYPERLINK(""./Eingangsrechnungen/SC/RE20038159...","=HYPERLINK(""./Eingangsrechnungen/SC/RE20038159..."
3,RE20038159,7671252,1.0,7.90,"Nadelset -BGM PRO- Dellorto PHBG (W3, W4, W5, ...",SC,2020-05-05,2021-12-15,0.19,Kleinmaterial,1.5010,9.40,7.90,1.50,"=HYPERLINK(""./Eingangsrechnungen/SC/RE20038159...","=HYPERLINK(""./Eingangsrechnungen/SC/RE20038159..."
4,RE20038159,7671257,1.0,6.97,Nebendüsenset -BGM PRO für Dellorto (Typ: 1488...,SC,2020-05-05,2021-12-15,0.19,Kleinmaterial,1.3243,8.29,6.97,1.32,"=HYPERLINK(""./Eingangsrechnungen/SC/RE20038159...","=HYPERLINK(""./Eingangsrechnungen/SC/RE20038159..."


In [20]:
movements["Bemerkung"] = ""

In [21]:
# print the sum of the column "Gesamtposition Netto"
print(f"Sum of Gesamtposition Netto: {movements['Gesamtposition Netto'].sum()}")

Sum of Gesamtposition Netto: 30676.89


In [22]:
# query all invoices which have 0 items and save them into a dataframe

if not quarter:
    zero_items = pd.read_sql_query(f"""
        SELECT r.[RechnungsNummer], r.[Art], r.[BetragNetto], r.[BetragBrutto], r.[Rechnungsdatum], r.[Zahlungsdatum], r.[Versandkosten], r.[Lieferant]
            FROM [EnderleSolutions].[dbo].[Rechnung_Lieferant] r
            WHERE r.RechnungsNummer NOT IN (SELECT w.RechnungsNummer FROM [EnderleSolutions].[dbo].[Wareneingang_Artikel] w)
            AND r.Zahlungsdatum >= '{start}' AND r.Zahlungsdatum <= '{end}';
    """, conn)

else:
    zero_items = pd.read_sql_query(f"""
        SELECT r.[RechnungsNummer], r.[Art], r.[BetragNetto], r.[BetragBrutto], r.[Rechnungsdatum], r.[Zahlungsdatum], r.[Versandkosten], r.[Lieferant]
            FROM [EnderleSolutions].[dbo].[Rechnung_Lieferant] r
            WHERE r.RechnungsNummer NOT IN (SELECT w.RechnungsNummer FROM [EnderleSolutions].[dbo].[Wareneingang_Artikel] w)
            AND r.Rechnungsdatum >= '{start}' AND r.Rechnungsdatum <= '{end}';
    """, conn)

zero_items.head(1)

C:\Users\Moritz\AppData\Local\Temp\ipykernel_7220\2745025011.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  zero_items = pd.read_sql_query(f"""


,RechnungsNummer,Art,BetragNetto,BetragBrutto,Rechnungsdatum,Zahlungsdatum,Versandkosten,Lieferant
0,0011-000017-3438,46 Miete,136.97,163.0,2021-08-16,2021-08-18,0.0,Hornbach


In [23]:
["RechnungsNummer", "Bezeichnung", "Verkaufspreis", "Steuersatz", "MwSt. pro Stück", "Anzahl", "Gesamtposition Netto", "MwSt. Gesamt", "Gesamtposition Brutto"]

['RechnungsNummer',
 'Bezeichnung',
 'Verkaufspreis',
 'Steuersatz',
 'MwSt. pro Stück',
 'Anzahl',
 'Gesamtposition Netto',
 'MwSt. Gesamt',
 'Gesamtposition Brutto']

In [24]:
# make a new column Bezeichnung which is all empty string
zero_items["Bezeichnung"] = zero_items["Lieferant"]
zero_items["Verkaufspreis"] = None
zero_items["Steuersatz"] = (zero_items["BetragBrutto"] - zero_items["BetragNetto"]) / zero_items["BetragNetto"]
zero_items["Steuersatz"] = zero_items["Steuersatz"].round(2)
zero_items["MwSt. pro Stück"] = None
zero_items["Anzahl"] = None
zero_items["Gesamtposition Netto"] = zero_items["BetragNetto"]
zero_items["MwSt. Gesamt"] = zero_items["BetragBrutto"] - zero_items["BetragNetto"]
zero_items["Gesamtposition Brutto"] = zero_items["BetragBrutto"]
zero_items["Bemerkung"] = "Rechnung ohne Artikel"
zero_items.head(4)

,RechnungsNummer,Art,BetragNetto,BetragBrutto,Rechnungsdatum,Zahlungsdatum,Versandkosten,Lieferant,Bezeichnung,Verkaufspreis,Steuersatz,MwSt. pro Stück,Anzahl,Gesamtposition Netto,MwSt. Gesamt,Gesamtposition Brutto,Bemerkung
0,0011-000017-3438,46 Miete,136.97,163.00,2021-08-16,2021-08-18,0.0,Hornbach,Hornbach,None,0.19,None,None,136.97,26.03,163.00,Rechnung ohne Artikel
1,1121,55 Beiträge,50.00,50.00,2021-01-01,2021-08-23,0.0,Vespa Veteranen-Club Deutschland,Vespa Veteranen-Club Deutschland,None,0.00,None,None,50.00,0.00,50.00,Rechnung ohne Artikel
2,1148,52 Recht und StB,600.25,714.30,2021-12-03,2021-12-22,0.0,Ressel und Ressel,Ressel und Ressel,None,0.19,None,None,600.25,114.05,714.30,Rechnung ohne Artikel
3,20210129-BW087-00008375913,55 Beiträge,30.20,35.94,2021-01-29,2021-01-29,0.0,Sparkasse Lörrach Rheinfelden,Sparkasse Lörrach Rheinfelden,None,0.19,None,None,30.20,5.74,35.94,Rechnung ohne Artikel


In [25]:
# create a column "Rechnung" which is the path to the invoice
zero_items["Rechnung"] = zero_items.apply(lambda x: paths(x["Rechnungsdatum"].year, str(x["Lieferant"]), str(x["RechnungsNummer"]))[0], axis=1)
# create a column "Zahlung" which is the path to the payment
zero_items["Zahlung"] = zero_items.apply(lambda x: paths(x["Rechnungsdatum"].year, str(x["Lieferant"]), str(x["RechnungsNummer"]))[1], axis=1)

In [26]:
# print the sum of the column "BetragNetto"
print(f"Sum of BetragNetto: {zero_items['BetragNetto'].sum()}")

Sum of BetragNetto: 2908.06


In [27]:
# get all unique values of the column Art
arten = list(set(list(movements["Art"].unique()) + list(zero_items["Art"].unique())))
arten

['52 Recht und StB',
 '57 Porto/Büro/Literatur',
 '46 Miete',
 'Kleinmaterial',
 '26 Wareneinkauf',
 '49 Kommunikation',
 '55 Beiträge']

In [28]:
writer = pd.ExcelWriter(path + 'Buchungskonten_Detailliert.xlsx', engine='xlsxwriter')

workbook  = writer.book

link_format = workbook.add_format({'num_format': '#,##0.00',
                                   'font_color': 'blue',
                                   'underline':  1,
                                   'align': 'center',})
percent_format = workbook.add_format({'num_format': '0%'})
euro_format = workbook.add_format({'num_format': '#,##0.00€'})
euro_format_bold = workbook.add_format({'num_format': '#,##0.00€', 'bold': True})

sums = 0

for art in arten:
    # create a new df thats a deep copy of the movements df
    df1 = movements.copy(deep=True)
    # filter the df where Art is arten[0]
    df1 = df1[df1["Art"] == art]
    # drop columns ArtikelNummer, Rechnungsdatum, Zahlungsdatum, Art
    df1 = df1.drop(columns=["ArtikelNummer", "Rechnungsdatum", "Zahlungsdatum", "Art"])
    # rearrange the columns
    df1 = df1[["RechnungsNummer", "Bezeichnung", "Verkaufspreis", "Steuersatz", "MwSt. pro Stück", "Anzahl", "Gesamtposition Netto", "MwSt. Gesamt", "Gesamtposition Brutto", "Rechnung", "Zahlung", "Bemerkung"]]

    df2 = zero_items.copy(deep=True)
    df2 = df2[df2["Art"] == art]
    df2 = df2[["RechnungsNummer", "Bezeichnung", "Verkaufspreis", "Steuersatz", "MwSt. pro Stück", "Anzahl", "Gesamtposition Netto", "MwSt. Gesamt", "Gesamtposition Brutto", "Rechnung", "Zahlung", "Bemerkung"]]

    df = pd.concat([df1, df2])

    sums += df["Gesamtposition Netto"].sum()

    # set the Steuersatz column to "keine" if it is 0
    df["Steuersatz"] = df["Steuersatz"].apply(lambda x: "" if x == 0 else x)
    df["MwSt. pro Stück"] = df["MwSt. pro Stück"].apply(lambda x: "" if x == 0 else x)
    df["MwSt. Gesamt"] = df["MwSt. Gesamt"].apply(lambda x: "" if x == 0 else x)

    name = art.replace("ä", "ae").replace("ö", "oe").replace("ü", "ue").replace("ß", "ss").replace("/", "_")
    df.to_excel(writer ,index=False, sheet_name=name)
    sheet = writer.sheets[name]

    # set the format of the column "Rechnung" to a link
    sheet.set_column('J:J', 8.2, link_format)
    # set the format of the column "Zahlung" to a link
    sheet.set_column('K:K', 8.2, link_format)
    # set the format of the column "Steuersatz" to a percent
    sheet.set_column('D:D', 8.7, percent_format)
    # set the format of the column "Gesamtposition Netto" to a euro
    sheet.set_column('G:G', 18.3, euro_format)
    # set the format of the column "MwSt. Gesamt" to a euro
    sheet.set_column('H:H', 14, euro_format)
    # set the format of the column "Gesamtposition Brutto" to a euro
    sheet.set_column('I:I', 18.8, euro_format)
    # set the format of the column "Verkaufspreis" to a euro
    sheet.set_column('C:C', 12, euro_format)
    # set the format of the column "MwSt. pro Stück" to a euro
    sheet.set_column('E:E', 13.6, euro_format)
    # set width of the column "RechnungsNummer
    sheet.set_column('A:A', 35)
    # set width of the column "Bezeichnung"
    sheet.set_column('B:B', 70)
    # set width of the column "Bemerkung"
    sheet.set_column('L:L', 24)

    # at the end of each euro column add a formula which sums the column
    sheet.write_formula(f"G{df.shape[0] + 3}", f"=SUM(G2:G{df.shape[0] + 1})", euro_format_bold)
    sheet.write_formula(f"H{df.shape[0] + 3}", f"=SUM(H2:H{df.shape[0] + 1})", euro_format_bold)
    sheet.write_formula(f"I{df.shape[0] + 3}", f"=SUM(I2:I{df.shape[0] + 1})", euro_format_bold)
    sheet.write_formula(f"C{df.shape[0] + 3}", f"=SUM(C2:C{df.shape[0] + 1})", euro_format_bold)
    sheet.write_formula(f"E{df.shape[0] + 3}", f"=SUM(E2:E{df.shape[0] + 1})", euro_format_bold)

writer.save()
writer.close()

sums


C:\Users\Moritz\AppData\Local\Temp\ipykernel_7220\3845381972.py:72: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
l:\My Drive\1EnderleSolutions\PYHTON\IN PROGRESS\Skript\venv2\Lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


33584.95

# Ein und ausgaben

In [29]:
writer = pd.ExcelWriter(path + 'Einnahmen_Ausgaben.xlsx', engine='xlsxwriter')

workbook  = writer.book

link_format = workbook.add_format({'num_format': '#,##0.00',
                                   'font_color': 'blue',
                                   'underline':  1,
                                   'align': 'center',})
percent_format = workbook.add_format({'num_format': '0%'})
euro_format = workbook.add_format({'num_format': '#,##0.00€'})
euro_format_bold = workbook.add_format({'num_format': '#,##0.00€', 'bold': True})

In [30]:
if not quarter:
    query = f"""
    SELECT * FROM [EnderleSolutions].[dbo].[Rechnung_Lieferant] r
    WHERE r.Zahlungsdatum >= '{start}' AND r.Zahlungsdatum <= '{end}';
    """

else:
    query = f"""
    SELECT * FROM [EnderleSolutions].[dbo].[Rechnung_Lieferant] r
    WHERE r.Rechnungsdatum >= '{start}' AND r.Rechnungsdatum <= '{end}';
    """

invoices = pd.read_sql(query, conn)
invoices.head()

C:\Users\Moritz\AppData\Local\Temp\ipykernel_7220\3605549877.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  invoices = pd.read_sql(query, conn)


,RechnungsNummer,Rechnungsdatum,Zahlungsdatum,Lieferant,BetragNetto,BetragBrutto,Bemerkung,Art,Kasse,RechnungsbelegDa,ZahlungsbelegDa,Link,Versandkosten,SSMA_TimeStamp,geprueft
0,0011-000017-3438,2021-08-16,2021-08-18,Hornbach,136.97,163.00,Pfand und Gasflaschenfüllung Argon,46 Miete,False,False,False,None,0.00,b'\x00\x00\x00\x00\x00\x13\xfdq',None
1,098-2020,2020-12-22,2021-02-15,Hausnummer12,568.97,660.00,None,None,False,False,False,None,0.00,b'\x00\x00\x00\x00\x00\x0f\xcfI',True
2,100485R1,2021-08-24,2021-10-13,Steckerladen,201.91,240.27,,None,False,False,False,None,6.90,b'\x00\x00\x00\x00\x00\x13\xc6\xab',None
3,103150,2021-03-23,2021-03-25,Hönert,103.95,123.70,,None,False,False,False,None,4.95,b'\x00\x00\x00\x00\x00\x13A\xbe',None
4,107125-2021,2021-05-26,2021-06-02,SIP,73.04,86.84,,None,False,False,False,None,1.26,b'\x00\x00\x00\x00\x00\x13Q\xed',None


In [31]:
invoices["MwSt."] = invoices["BetragBrutto"] - invoices["BetragNetto"]
invoices = invoices[["RechnungsNummer", "Rechnungsdatum", "Zahlungsdatum", "Lieferant", "BetragNetto", "MwSt.", "BetragBrutto", "Kasse"]]
invoices.head(5)

,RechnungsNummer,Rechnungsdatum,Zahlungsdatum,Lieferant,BetragNetto,MwSt.,BetragBrutto,Kasse
0,0011-000017-3438,2021-08-16,2021-08-18,Hornbach,136.97,26.03,163.00,False
1,098-2020,2020-12-22,2021-02-15,Hausnummer12,568.97,91.03,660.00,False
2,100485R1,2021-08-24,2021-10-13,Steckerladen,201.91,38.36,240.27,False
3,103150,2021-03-23,2021-03-25,Hönert,103.95,19.75,123.70,False
4,107125-2021,2021-05-26,2021-06-02,SIP,73.04,13.80,86.84,False


In [32]:
# create a column "Rechnung" which is the path to the invoice
invoices["Rechnung"] = invoices.apply(lambda x: paths(x["Rechnungsdatum"].year, str(x["Lieferant"]), str(x["RechnungsNummer"]))[0], axis=1)
# create a column "Zahlung" which is the path to the payment
invoices["Zahlung"] = invoices.apply(lambda x: paths(x["Rechnungsdatum"].year, str(x["Lieferant"]), str(x["RechnungsNummer"]))[1], axis=1)

In [33]:
# set date format for the columns "Rechnungsdatum" and "Zahlungsdatum" to dd.mm.yyyy
invoices["Rechnungsdatum"] = invoices["Rechnungsdatum"].dt.strftime("%d.%m.%Y")
invoices["Zahlungsdatum"] = invoices["Zahlungsdatum"].dt.strftime("%d.%m.%Y")

In [34]:
# set Kasse to "" if False else to "Ja"
invoices["Kasse"] = invoices["Kasse"].apply(lambda x: "" if x == False else "Ja")

In [35]:
invoices.to_excel(writer ,index=False, sheet_name="Ausgaben")
sheet = writer.sheets["Ausgaben"]

# add sums at the end
sheet.write_formula(f"F{invoices.shape[0] + 3}", f"=SUM(F2:F{invoices.shape[0] + 1})", euro_format_bold)
sheet.write_formula(f"E{invoices.shape[0] + 3}", f"=SUM(E2:E{invoices.shape[0] + 1})", euro_format_bold)
sheet.write_formula(f"G{invoices.shape[0] + 3}", f"=SUM(G2:G{invoices.shape[0] + 1})", euro_format_bold)


sheet.set_column('J:J', 8.2, link_format)
sheet.set_column('I:I', 8.2, link_format)
sheet.set_column('F:F', 14, euro_format)
sheet.set_column('E:E', 18.3, euro_format)
sheet.set_column('G:G', 18.8, euro_format)
sheet.set_column('A:A', 25)
sheet.set_column('D:D', 30)
sheet.set_column('B:B', 15)
sheet.set_column('C:C', 15)


0

In [36]:
if not quarter:
    query = f"""
    SELECT * FROM [EnderleSolutions].[dbo].[Rechnung_Kunde] r
    WHERE r.Zahlungsdatum >= '{start}' AND r.Zahlungsdatum <= '{end}';
    """

else:
    query = f"""
    SELECT * FROM [EnderleSolutions].[dbo].[Rechnung_Kunde] r
    WHERE r.Datum >= '{start}' AND r.Datum <= '{end}';
    """

out_invoices = pd.read_sql(query, conn)
out_invoices.head()

C:\Users\Moritz\AppData\Local\Temp\ipykernel_7220\261124722.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  out_invoices = pd.read_sql(query, conn)


,Rechnungsnummer,Datum,KundenNummer,Betrag,Versandart,Versandkosten,Kasse,Zahlungsdatum,Steuer
0,001.21,2021-01-03,146,209.29,DHL Paket 2 kg,4.99,False,2021-01-08,0
1,002.21,2021-01-17,150,73.99,DHL Paket 2 kg,4.99,False,2021-01-20,0
2,003.21,2021-01-17,151,321.99,DHL Paket 2 kg,4.99,False,2021-02-01,0
3,004.21,2021-01-17,117,14.35,Großbrief,1.55,False,2021-01-21,0
4,005.21,2021-01-17,152,64.00,Abholung,0.00,False,2021-01-26,0


In [37]:
out_invoices["Netto"] = out_invoices["Betrag"] / (1 + out_invoices["Steuer"] / 100)
out_invoices["MwSt."] = out_invoices["Betrag"] - out_invoices["Netto"]

In [38]:
out_invoices = out_invoices[["Rechnungsnummer", "Datum", "Zahlungsdatum", "KundenNummer", "Netto", "MwSt.", "Betrag", "Kasse"]]
out_invoices = out_invoices.rename(columns={"Rechnungsnummer": "RechnungsNummer", "Datum": "Rechnungsdatum", "Zahlungsdatum": "Zahlungsdatum", "KundenNummer": "Kunden Nr.", "Betrag": "Brutto", "Kasse": "Kasse"})
out_invoices.head()

,RechnungsNummer,Rechnungsdatum,Zahlungsdatum,Kunden Nr.,Netto,MwSt.,Brutto,Kasse
0,001.21,2021-01-03,2021-01-08,146,209.29,0.0,209.29,False
1,002.21,2021-01-17,2021-01-20,150,73.99,0.0,73.99,False
2,003.21,2021-01-17,2021-02-01,151,321.99,0.0,321.99,False
3,004.21,2021-01-17,2021-01-21,117,14.35,0.0,14.35,False
4,005.21,2021-01-17,2021-01-26,152,64.00,0.0,64.00,False


In [39]:
def exist_out(inv_num) -> tuple:
    return (
        os.path.exists(f"../../../DATEN/Ausgangsrechnungen/Druck/{inv_num}.pdf"),
        os.path.exists(f"../../../DATEN/Ausgangsrechnungen/Zahlungen/{inv_num}.pdf"),
    )

def paths_out(inv_num):
    return (
        f'=HYPERLINK("./Ausgangsrechnungen/{inv_num}/rechnung.pdf", "öffnen")' if exist_out(inv_num)[0] else "---",
        f'=HYPERLINK("./Ausgangsrechnungen/{inv_num}/zahlung.pdf", "öffnen")' if exist_out(inv_num)[1] else "---",
    )

In [40]:
out_invoices["Rechnung"] = out_invoices["RechnungsNummer"].apply(lambda x: paths_out(x)[0])
out_invoices["Zahlung"] = out_invoices["RechnungsNummer"].apply(lambda x: paths_out(x)[1])
out_invoices.head()

,RechnungsNummer,Rechnungsdatum,Zahlungsdatum,Kunden Nr.,Netto,MwSt.,Brutto,Kasse,Rechnung,Zahlung
0,001.21,2021-01-03,2021-01-08,146,209.29,0.0,209.29,False,"=HYPERLINK(""./Ausgangsrechnungen/001.21/rechnu...","=HYPERLINK(""./Ausgangsrechnungen/001.21/zahlun..."
1,002.21,2021-01-17,2021-01-20,150,73.99,0.0,73.99,False,"=HYPERLINK(""./Ausgangsrechnungen/002.21/rechnu...","=HYPERLINK(""./Ausgangsrechnungen/002.21/zahlun..."
2,003.21,2021-01-17,2021-02-01,151,321.99,0.0,321.99,False,"=HYPERLINK(""./Ausgangsrechnungen/003.21/rechnu...","=HYPERLINK(""./Ausgangsrechnungen/003.21/zahlun..."
3,004.21,2021-01-17,2021-01-21,117,14.35,0.0,14.35,False,"=HYPERLINK(""./Ausgangsrechnungen/004.21/rechnu...","=HYPERLINK(""./Ausgangsrechnungen/004.21/zahlun..."
4,005.21,2021-01-17,2021-01-26,152,64.00,0.0,64.00,False,"=HYPERLINK(""./Ausgangsrechnungen/005.21/rechnu...","=HYPERLINK(""./Ausgangsrechnungen/005.21/zahlun..."


In [41]:
# convert to string dd.mm.yyyy
out_invoices["Rechnungsdatum"] = out_invoices["Rechnungsdatum"].apply(lambda x: x.strftime("%d.%m.%Y") if not pd.isnull(x) else "---")
out_invoices["Zahlungsdatum"] = out_invoices["Zahlungsdatum"].apply(lambda x: x.strftime("%d.%m.%Y") if not pd.isnull(x) else "---")

In [42]:
out_invoices["Kasse"] = out_invoices["Kasse"].apply(lambda x: "" if x == False else "Ja")

In [43]:
out_invoices.to_excel(writer ,index=False, sheet_name="Einnahmen")
sheet = writer.sheets["Einnahmen"]

# make the sums at the bottom of the sheet
sheet.write("E" + str(len(out_invoices) + 2), "=SUM(E2:E" + str(len(out_invoices) + 1) + ")", euro_format_bold)

sheet.set_column('A:A', 12)
sheet.set_column('B:B', 15)
sheet.set_column('C:C', 15)
sheet.set_column('D:D', 12.5)
sheet.set_column('E:E', 12.5, euro_format)
sheet.set_column('F:F', 12.5, euro_format)
sheet.set_column('G:G', 12.5, euro_format)
sheet.set_column('H:H', 12.5)
sheet.set_column('I:I', 12.5, link_format)
sheet.set_column('J:J', 12.5, link_format)

0

In [44]:
writer.close()

In [45]:
import shutil

if not os.path.exists(path + "Eingangsrechnungen"):
    os.makedirs(path + "Eingangsrechnungen")

for inv in invoices.values:
    year = inv[1].split(".")[-1]
    inv_num = inv[0]
    invoicer = inv[3]
    inv_path = f"/{invoicer.replace(r'&', '_')}/{inv_num.replace(r'&', '_')}/"
    if not os.path.exists(path + f"Eingangsrechnungen{inv_path}"):
        os.makedirs(path + f"Eingangsrechnungen{inv_path}")

    if os.path.exists(f"../../../DATEN/Eingangsrechnungen/{year}/{invoicer.replace(r'&', '_')}/{inv_num.replace(r'&', '_')}/rechnung.pdf"):
        shutil.copy(f"../../../DATEN/Eingangsrechnungen/{year}/{invoicer.replace(r'&', '_')}/{inv_num.replace(r'&', '_')}/rechnung.pdf",
                    path + f"Eingangsrechnungen{inv_path}rechnung.pdf"
        )
    if os.path.exists(f"../../../DATEN/Eingangsrechnungen/{year}/{invoicer.replace(r'&', '_')}/{inv_num.replace(r'&', '_')}/zahlung.pdf"):
        shutil.copy(f"../../../DATEN/Eingangsrechnungen/{year}/{invoicer.replace(r'&', '_')}/{inv_num.replace(r'&', '_')}/zahlung.pdf",
                    path + f"Eingangsrechnungen{inv_path}zahlung.pdf"
        )

In [46]:
if not os.path.exists(path + f"Ausgangsrechnungen"):
    os.makedirs(path + f"Ausgangsrechnungen")

for inv in out_invoices.values:
    inv_num = inv[0]
    inv_path = f"/{inv_num}/"

    if not os.path.exists(path + f"Ausgangsrechnungen{inv_path}"):
        os.makedirs(path + f"Ausgangsrechnungen{inv_path}")

    if os.path.exists(f"../../../DATEN/Ausgangsrechnungen/Druck/{inv_num}.pdf"):
        shutil.copy(f"../../../DATEN/Ausgangsrechnungen/Druck/{inv_num}.pdf",
                    path + f"Ausgangsrechnungen{inv_path}rechnung.pdf"
    )
    if os.path.exists(f"../../../DATEN/Ausgangsrechnungen/Zahlungen/{inv_num}.pdf"):
        shutil.copy(f"../../../DATEN/Ausgangsrechnungen/Zahlungen/{inv_num}.pdf",
                    path + f"Ausgangsrechnungen{inv_path}zahlung.pdf"
    )

In [47]:
len(invoices[invoices["Lieferant"] == "SC"])

43